In [3]:
import os
import cv2
import mediapipe as mp
import numpy as np
from tqdm import tqdm

# === Settings ===
train_dir = 'filtered_train'
test_dir = 'filtered_test'
output_folder = 'keypoints_top20_84'  # output folder for npy files
frames_per_video = 30
features_per_frame = 84  # 2 coordinates (x, y) × 21 landmarks × 2 hands

# === Initialize Mediapipe Hands only ===
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2)

# === Helper: extract only (x, y) from both hands ===
def extract_hand_keypoints(image):
    result = hands.process(image)
    keypoints = []

    # Initialize empty hands
    left_hand = None
    right_hand = None

    if result.multi_hand_landmarks and result.multi_handedness:
        for idx, hand_landmarks in enumerate(result.multi_hand_landmarks):
            handedness = result.multi_handedness[idx].classification[0].label
            if handedness == "Left":
                left_hand = hand_landmarks
            elif handedness == "Right":
                right_hand = hand_landmarks

    # Extract left hand
    if left_hand:
        for lm in left_hand.landmark:
            keypoints.extend([lm.x, lm.y])
    else:
        keypoints.extend([0] * 21 * 2)

    # Extract right hand
    if right_hand:
        for lm in right_hand.landmark:
            keypoints.extend([lm.x, lm.y])
    else:
        keypoints.extend([0] * 21 * 2)

    return keypoints

# === Helper: process a folder (train/test) ===
def process_folder(folder_path):
    X = []
    y = []
    class_labels = os.listdir(folder_path)
    label_to_index = {label: idx for idx, label in enumerate(sorted(class_labels))}

    for label in tqdm(class_labels, desc=f"Processing {folder_path}"):
        class_folder_path = os.path.join(folder_path, label)
        if not os.path.isdir(class_folder_path):
            continue

        for video_name in os.listdir(class_folder_path):
            if not video_name.endswith(('.mp4', '.avi', '.mov', '.mkv', '.webm')):
                continue

            video_path = os.path.join(class_folder_path, video_name)
            cap = cv2.VideoCapture(video_path)
            if not cap.isOpened():
                print(f"⚠️ Skipping unreadable video: {video_path}")
                continue

            frames = []
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            frame_interval = max(1, total_frames // frames_per_video)

            frame_idx = 0
            while cap.isOpened() and len(frames) < frames_per_video:
                ret, frame = cap.read()
                if not ret:
                    break

                if frame_idx % frame_interval == 0:
                    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    keypoints = extract_hand_keypoints(frame_rgb)
                    frames.append(keypoints)

                frame_idx += 1

            cap.release()

            # Only save sample if frames are complete and consistent
            if len(frames) == frames_per_video and all(len(f) == features_per_frame for f in frames):
                X.append(frames)
                y.append(label_to_index[label])
            else:
                print(f"⚠️ Skipping incomplete/broken video: {video_path}")

    return np.array(X), np.array(y), label_to_index

# === Run extraction ===
os.makedirs(output_folder, exist_ok=True)

# Process train
x_train, y_train, label_map_train = process_folder(train_dir)
np.save(os.path.join(output_folder, 'x_train.npy'), x_train)
np.save(os.path.join(output_folder, 'y_train.npy'), y_train)

# Process test
x_test, y_test, label_map_test = process_folder(test_dir)
np.save(os.path.join(output_folder, 'x_test.npy'), x_test)
np.save(os.path.join(output_folder, 'y_test.npy'), y_test)

# Save label map
import json
with open(os.path.join(output_folder, 'label_map.json'), 'w') as f:
    json.dump(label_map_train, f)

print("\n✅ Keypoint extraction complete!")
print(f"Saved files to: {output_folder}/")
print(f"x_train shape: {x_train.shape}, y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape}, y_test shape: {y_test.shape}")


I0000 00:00:1745764697.535436   29619 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1 Pro
Processing filtered_train:   0%|                         | 0/20 [00:00<?, ?it/s]W0000 00:00:1745764697.545491   52887 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745764697.553618   52887 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


⚠️ Skipping incomplete/broken video: filtered_train/again/nvaS2J5mcEk_030.mp4
⚠️ Skipping incomplete/broken video: filtered_train/again/gU5EI_ZTzxw_021.mp4
⚠️ Skipping incomplete/broken video: filtered_train/again/UI5KLVn8JKI_007.mp4


Processing filtered_train:   5%|▊                | 1/20 [00:28<08:59, 28.40s/it]

⚠️ Skipping incomplete/broken video: filtered_train/friend/YasOpK1RNvg_011.mp4
⚠️ Skipping incomplete/broken video: filtered_train/friend/pv1yeA4iHmU_016.mp4


Processing filtered_train:  25%|████▎            | 5/20 [02:27<07:31, 30.11s/it]

...

⚠️ Skipping incomplete/broken video: filtered_test/drink/OSPBKnsTuDY_012.mp4


Processing filtered_test:  95%|████████████████▏| 19/20 [03:51<00:12, 12.62s/it]

⚠️ Skipping incomplete/broken video: filtered_test/orange/xEw0KrlJA6Q_029.mp4


Processing filtered_test: 100%|█████████████████| 20/20 [04:04<00:00, 12.20s/it]


✅ Keypoint extraction complete!
Saved files to: keypoints_top20_84/
x_train shape: (634, 30, 84), y_train shape: (634,)
x_test shape: (252, 30, 84), y_test shape: (252,)


In [4]:
import numpy as np
import json
import pandas as pd

# === Load saved files ===
y_train = np.load('keypoints_top20_84/y_train.npy')
y_test = np.load('keypoints_top20_84/y_test.npy')

with open('keypoints_top20_84/label_map.json', 'r') as f:
    label_map = json.load(f)

# === Reverse the label_map
index_to_label = {v: k for k, v in label_map.items()}

# === Count frequencies
train_class_counts = pd.Series(y_train).map(index_to_label).value_counts()
test_class_counts = pd.Series(y_test).map(index_to_label).value_counts()

# === Show results
print("\n📊 Training set class frequencies:")
print(train_class_counts)

print("\n📊 Test set class frequencies:")
print(test_class_counts)

# === Save to CSV if needed
train_class_counts.to_csv('train_class_frequencies_after_extraction.csv')
test_class_counts.to_csv('test_class_frequencies_after_extraction.csv')

print("\n✅ Class frequencies saved to CSV files!")



📊 Training set class frequencies:
fish       37
milk       37
eat        37
nice       35
teacher    33
cousin     33
sister     32
water      32
student    32
white      32
yes        31
finish     30
orange     30
want       30
what       29
friend     29
school     29
no         29
drink      29
again      28
Name: count, dtype: int64

📊 Test set class frequencies:
fish       14
nice       14
white      14
milk       14
finish     13
want       13
school     13
eat        13
orange     13
no         13
water      13
student    12
cousin     12
yes        12
teacher    12
sister     12
drink      12
friend     11
what       11
again      11
Name: count, dtype: int64

✅ Class frequencies saved to CSV files!
